In [129]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
from nltk import ne_chunk
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf
import re
import heapq

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [130]:
from google.colab import drive
drive.mount('/gdrive')

f =open('/gdrive/My Drive/Colab_ML/Abstractive Summarizer/convotext.txt','r').read().lower()
with tf.device('/device:GPU:0'):
#     f = re.sub(r'\s+', ' ', f)  
    no_of_lines = len(open('/gdrive/My Drive/Colab_ML/Abstractive Summarizer/convotext.txt','r').readlines())
    stop_words = set(stopwords.words('english'))


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# ! cd "/gdrive/My Drive/Colab_ML/Abstractive Summarizer" && wget "https://conceptnet.s3.amazonaws.com/downloads/2017/numberbatch/numberbatch-en-17.06.txt.gz"
# ! cd "/gdrive/My Drive/Colab_ML/Abstractive Summarizer/" && gunzip "/gdrive/My Drive/Colab_ML/Abstractive Summarizer/numberbatch-en-17.06.txt.gz"

In [ ]:
def remove_punc(sent):
    with tf.device('/device:GPU:0'):
        punctuations = '''!()-[]{};'"\,<>./?@#%^&*_~'''
        for x in sent: 
            if x in punctuations: 
                sent = sent.replace(x, "")
        return sent

In [ ]:
def preprocess(sent):
    with tf.device('/device:GPU:0'):
        sent = remove_punc(sent)
        sent = nltk.word_tokenize(sent,language='english')
        lemmatizer = WordNetLemmatizer()
        sent = [lemmatizer.lemmatize(x) for x in sent]
        sent = ' '.join(sent)
        filtered_sentence = [w for w in sent.split(' ') if not w in stop_words] 
                
        return ' '.join(filtered_sentence)

In [ ]:
def weighted_freq(sent):
    word_frequencies = {}  
    for word in sent:  
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
    
    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    return word_frequencies

In [ ]:
def sent_score_calc(text,word_frequencies):
    sentence_list = nltk.sent_tokenize(text)
    sentence_scores = {}  
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 10:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sesnt] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    return sentence_scores
    

In [136]:
with tf.device('/device:GPU:0'):
    nlp = en_core_web_sm.load()
    docu = preprocess(f)
    doc = nlp(docu)
#     pprint([(X.text, X.label_) for X in doc.ents])
#     pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])
#     displacy.render((doc), jupyter=True, style='ent')
    max_freq=weighted_freq(docu)
    sent_scores = sent_score_calc(f,max_freq)
    summary_sentences = heapq.nlargest(int(no_of_lines/2), sent_scores, key=sent_scores.get)

    summary = ' '.join(summary_sentences)
    print(summary.capitalize()) 

As i said, i'll call you back. tim:	ah, it’s just me. dolores:	yes, sure, i understand. dolores:	
can i help you with anything else? tim:	and is there a discount rate for conference delegates? can i book that, then? i just need to check one or two details. can i help you with anything else? dolores:	certainly. dolores:	no, tax is another $70 on top of that. dolores:	hello! dolores speaking…
tim:	ah yes, hello. tim:	and how much will that be?
